In [11]:
import os
import pandas as pd
from typing import List, Dict

In [12]:

def analyze_stock_price_moves(file_path: str, window_size: int = 21, min_move: float = 0.5) -> List[Dict]:
    """
    Analyze stock price moves within a given window size and minimum move percentage.
    
    Parameters:
    - file_path (str): The path to the stock data CSV file.
    - window_size (int): The window size for calculating price moves. Default is 21 days.
    - min_move (float): The minimum price move (as a percentage). Default is 0.5 (i.e., 50%).
    
    Returns:
    - List[Dict]: A list of dictionaries containing the start date, end date, high, low, and percentage move for each window.
    """
    # Load the data from the file
    data = pd.read_csv(file_path)
    
    # Initialize an empty list to store the results
    results = []
    
    # Initialize index i
    i = 0
    
    # Loop through the data
    while i <= len(data) - window_size:
        window_data = data.iloc[i:i + window_size]
        
        # Calculate high, low, and percentage move
        high_price = window_data['High'].max()
        low_price = window_data['Low'].min()
        
        if low_price == 0:
            i += 1
            continue
        
        perc_move = (high_price - low_price) / low_price
        
        # Check if the percentage move is greater than or equal to the minimum move
        if perc_move >= min_move:
            result = {
                'Start Date': window_data.iloc[0]['Date'],
                'End Date': window_data.iloc[-1]['Date'],
                'High': high_price,
                'Low': low_price,
                'Percentage Move': perc_move * 100  # convert to percentage
            }
            results.append(result)
            i += window_size  # Skip to the next window
        else:
            i += 1  # Move to the next day
    
    return results

def analyze_all_stocks(directory_path: str, window_size: int = 21, min_move: float = 0.5) -> Dict[str, List[Dict]]:
    """
    Analyze stock price moves for all stocks within a directory.
    
    Parameters:
    - directory_path (str): The path to the directory containing stock data CSV files.
    - window_size (int): The window size for calculating price moves. Default is 21 days.
    - min_move (float): The minimum price move (as a percentage). Default is 0.5 (i.e., 50%).
    
    Returns:
    - Dict[str, List[Dict]]: A dictionary where the key is the stock name and the value is a list of dictionaries containing
                              the start date, end date, high, low, and percentage move for each window.
    """
    # Initialize an empty dictionary to store the results for all stocks
    all_stock_results = {}
    
    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith('.csv'):
            print(f"Processing {filename}...")  # Print the stock being processed
            file_path = os.path.join(directory_path, filename)
            stock_name = filename.split('.')[0]
            
            # Analyze the stock and store the results
            stock_results = analyze_stock_price_moves(file_path, window_size, min_move)
            if stock_results:  # Only add to the dictionary if there are any significant moves
                all_stock_results[stock_name] = stock_results
    
    return all_stock_results

In [13]:

# Run the function for all stocks. Replace 'your_directory_path' with the actual directory path.
dir_path = "/Users/yash/Desktop/Trading/code/nse_data/eod2/src/eod2_data/daily"
all_stock_results = analyze_all_stocks(dir_path)

# Uncomment the above line and replace 'your_directory_path' with your actual directory path.


Processing intlconv.csv...
Processing diamondyd.csv...
Processing aeroflex.csv...
Processing bhartiartl.csv...
Processing infinium_sme.csv...
Processing priviscl.csv...
Processing gabriel.csv...
Processing avroind.csv...
Processing 3pland.csv...
Processing zentec.csv...
Processing tatasteel.csv...
Processing senco.csv...
Processing omfurn_sme.csv...
Processing blblimited.csv...
Processing relaxo.csv...
Processing infomedia.csv...
Processing lupin.csv...
Processing jitfinfra.csv...
Processing bfinvest.csv...
Processing nifty pharma.csv...
Processing boschltd.csv...
Processing superhouse.csv...
Processing fiemind.csv...
Processing nesco.csv...
Processing ahl.csv...
Processing lincoln.csv...
Processing ideaforge.csv...
Processing caplipoint.csv...
Processing patintlog.csv...
Processing banswras.csv...
Processing mkpl_sme.csv...
Processing yatra.csv...
Processing avonmore.csv...
Processing jktyre.csv...
Processing vipclothng.csv...
Processing welinv.csv...
Processing shriramfin.csv...
Proc

In [16]:
type(all_stock_results)

dict

In [26]:
import pandas as pd
from typing import List, Dict
import talib as ta

# Function to add Average Turnover
def add_average_turnover(stock_data: pd.DataFrame, results_dict: Dict, avg_vol_window: int = 50):
    avg_volume = ta.SMA(stock_data['Volume'].values.astype('double'), timeperiod=avg_vol_window)
    for entry in results_dict:
        if 'Average Turnover' in entry:
            continue
        start_date = entry['Start Date']
        start_index = stock_data.index[stock_data['Date'] == start_date][0]
        day_before_index = start_index - 1
        if day_before_index < avg_vol_window - 1:
            continue
        price = stock_data.iloc[day_before_index]['Close']
        avg_turnover = avg_volume[day_before_index] * price
        entry['Average Turnover'] = avg_turnover

# Function to add 20-Day ADR
def add_20_day_ADR(stock_data: pd.DataFrame, results_dict: Dict):
    adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)
    for entry in results_dict:
        if '20-Day ADR%' in entry:
            continue
        start_date = entry['Start Date']
        start_index = stock_data.index[stock_data['Date'] == start_date][0]
        day_before_index = start_index - 1
        if day_before_index < 19:
            continue
        entry['20-Day ADR%'] = adr[day_before_index]

# Function to add ATR (Average True Range)
def add_ATR(stock_data: pd.DataFrame, results_dict: Dict, atr_window: int = 14):
    atr = ta.ATR(stock_data['High'].values.astype('double'), stock_data['Low'].values.astype('double'), stock_data['Close'].values.astype('double'), timeperiod=atr_window)
    for entry in results_dict:
        if 'ATR' in entry:
            continue
        start_date = entry['Start Date']
        start_index = stock_data.index[stock_data['Date'] == start_date][0]
        day_before_index = start_index - 1
        if day_before_index < atr_window - 1:
            continue
        entry['ATR'] = atr[day_before_index]

# Function to add distances from EMAs (Exponential Moving Averages)
def add_distances_from_EMA(stock_data: pd.DataFrame, results_dict: Dict, ema_periods: List[int] = [10, 20, 50, 200]):
    for period in ema_periods:
        ema = ta.EMA(stock_data['Close'].values.astype('double'), timeperiod=period)
        ema_key = f'Distance from {period}EMA'
        for entry in results_dict:
            if ema_key in entry:
                continue
            start_date = entry['Start Date']
            start_index = stock_data.index[stock_data['Date'] == start_date][0]
            day_before_index = start_index - 1
            if day_before_index < period - 1:
                continue
            close_price = stock_data.iloc[day_before_index]['Close']
            distance_from_ema = close_price - ema[day_before_index]
            entry[ema_key] = distance_from_ema


In [27]:
for stock_name, moves in all_stock_results.items():
    stock_file_path = f"{dir_path}/{stock_name}.csv"
    
    # Check if the file exists; if not, skip to the next stock
    if not os.path.exists(stock_file_path):
        print(f"File for {stock_name} not found. Skipping.")
        continue

    print(f"Updating {stock_name}...")
    stock_data = pd.read_csv(stock_file_path)

    updated_moves = moves.copy()  # Create a copy to avoid in-place modifications
    add_average_turnover(stock_data, updated_moves)
    add_20_day_ADR(stock_data, updated_moves)
    add_ATR(stock_data, updated_moves)
    add_distances_from_EMA(stock_data, updated_moves)
    all_stock_results[stock_name] = updated_moves  # Explicitly update

Updating intlconv...
Updating diamondyd...
Updating bhartiartl...
Updating infinium_sme...
Updating priviscl...
Updating gabriel...
Updating avroind...
Updating 3pland...
Updating zentec...
Updating tatasteel...
Updating senco...
Updating omfurn_sme...
Updating blblimited...
Updating relaxo...
Updating infomedia...
Updating lupin...
Updating jitfinfra...
Updating bfinvest...
Updating nifty pharma...
Updating boschltd...
Updating superhouse...
Updating fiemind...
Updating nesco...
Updating ahl...
Updating lincoln...
Updating caplipoint...
Updating patintlog...
Updating banswras...
Updating mkpl_sme...
Updating jktyre...
Updating vipclothng...
Updating welinv...
Updating shriramfin...
Updating allcargo...
Updating shripiston...
Updating conti_sme...
Updating cinevista...
Updating v2retail...
Updating salasar...
Updating pearlpoly...
Updating bkmindst...
Updating headsup...
Updating rbminfra_sme...
Updating kirloseng...
Updating unienter...
Updating ginnifila...


/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_36150/1591755267.py:22: RuntimeWarning: invalid value encountered in divide
  adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)


Updating autoaxles...
Updating kesoramind...
Updating epigral...
Updating abcots_sme...
Updating pricolltd...
Updating zenithexpo...
Updating shalby...
Updating gisolution...
Updating star...
Updating mep...
Updating railtel...
Updating technoe...
Updating tdpowersys...
Updating manakcoat...
Updating chennpetro...
Updating onepoint...
Updating kck_sme...
Updating fcl...
Updating gepil...
Updating mitcon...
Updating qfil_sme...
Updating dycl...
Updating pcjeweller...
Updating prakash...


/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_36150/1591755267.py:22: RuntimeWarning: invalid value encountered in divide
  adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)


Updating ranasug...
Updating il&fsengg...
Updating capacite...
Updating hindnatgls...
Updating worth...
Updating hcc...
Updating nam-india...
Updating cgpower...
Updating nifty private bank...
Updating dynamic_sme...
Updating ksl...


...

Updating sci...
Updating andhrsugar...
Updating rajratan...
Updating heidelberg...
Updating til...
Updating vaishali...
Updating lotuseye...
Updating hindcopper...
Updating shoperstop...
Updating mindacorp...
Updating oil...
Updating aspinwall...
Updating hondapower...
Updating gicre...
Updating steelcity...
Updating oilcountub...
Updating mukandltd...
Updating trf...
Updating kore_sme...
Updating iitl...
Updating asiantiles...
Updating umangdairy...
Updating apcotexind...
Updating kitex...
Updating pidilitind...
Updating lumaxtech...
Updating carysil...
Updating indramedco...
Updating artemismed...
Updating vinny...
Updating pavnaind_sme...
Updating frog_sme...
Updating raymond...
Updating hpil...
Updating fmgoetze...
Updating castrolind...
Updating landmark...
Updating softtech...
Updating bmetrics_sme...
Updating uttamsugar...
Updating hleglas...
Updating dhanbank...
Updating yesbank...
Updating zuariind...


/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_36150/1591755267.py:22: RuntimeWarning: invalid value encountered in divide
  adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)


Updating satindltd...
Updating pashupati_sme...
Updating digjamlmtd...
Updating nagreekcap...
Updating jyotistruc...
Updating mdl_sme...
Updating beardsell...
Updating splil...
Updating jublinds...
Updating sadbhin...
Updating muthootcap...
Updating dabur...
Updating selan...
Updating hesterbio...
Updating hndfds...
Updating tfciltd...
Updating kohinoor...
Updating ameya_sme...
Updating trigyn...
Updating psb...
Updating narmada_sme...
Updating uniinfo...
Updating dwarkesh...
Updating elgirubco...
Updating medicameq...
Updating goldiam...
Updating amarajabat...
Updating abcapital...
Updating tipsfilms...
Updating dtil...
Updating shreepushk...
Updating jash...
Updating nilkamal...
Updating jswenergy...
Updating astrazen...
Updating dhampursug...
Updating starhealth...
Updating schand...
Updating arenterp...
Updating bluecoast...
Updating smlisuzu...
Updating vividha...
Updating kakatcem...
Updating shreyanind...
Updating swelectes...
Updating kotarisug...
Updating pfc...
Updating walpa

/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_36150/1591755267.py:22: RuntimeWarning: invalid value encountered in divide
  adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)
/var/folders/_3/qbdc2k351_379d2nk16_q9p80000gn/T/ipykernel_36150/1591755267.py:22: RuntimeWarning: invalid value encountered in divide
  adr = 100 * (ta.SMA(stock_data['High'].values.astype('double') / stock_data['Low'].values.astype('double'), timeperiod=20) - 1)


Updating khadim...
Updating svll...
Updating panamapet...
Updating persistent...
Updating ajmera...
Updating superspin...
Updating ucobank...
Updating vhl...
Updating bluestarco...
Updating cholahldng...
Updating kaya...
Updating avsl_sme...
Updating kansainer...
Updating treehouse...
Updating jakharia_sme...
Updating jetairways...
Updating veranda...
Updating beml...
Updating muthootfin...
Updating polymed...
Updating redington...
Updating mcdowell-n...
Updating isec...
Updating awhcl...
Updating iex...
Updating bconcepts...
Updating shrenik...
Updating tatastllp...
Updating centralbk...
Updating tanla...
Updating bhandari...
Updating ricoauto...
Updating quess...
Updating nifty realty...
Updating mangalam...
Updating rvhl...
Updating sptl...
Updating crayons_sme...
Updating mangchefer...
Updating ugarsugar...
Updating tirupati_sme...
Updating udaicement...
Updating ppl...
Updating prsmjohnsn...
Updating lovable...
Updating bewltd_sme...


In [30]:
all_stock_results.keys()

dict_keys(['intlconv', 'diamondyd', 'bhartiartl', 'infinium_sme', 'priviscl', 'gabriel', 'avroind', '3pland', 'zentec', 'tatasteel', 'senco', 'omfurn_sme', 'blblimited', 'relaxo', 'infomedia', 'lupin', 'jitfinfra', 'bfinvest', 'nifty pharma', 'boschltd', 'superhouse', 'fiemind', 'nesco', 'ahl', 'lincoln', 'caplipoint', 'patintlog', 'banswras', 'mkpl_sme', 'jktyre', 'vipclothng', 'welinv', 'shriramfin', 'allcargo', 'shripiston', 'conti_sme', 'cinevista', 'v2retail', 'salasar', 'pearlpoly', 'bkmindst', 'headsup', 'rbminfra_sme', 'kirloseng', 'unienter', 'ginnifila', 'autoaxles', 'kesoramind', 'epigral', 'abcots_sme', 'pricolltd', 'zenithexpo', 'shalby', 'gisolution', 'star', 'mep', 'railtel', 'technoe', 'tdpowersys', 'manakcoat', 'chennpetro', 'onepoint', 'kck_sme', 'fcl', 'gepil', 'mitcon', 'qfil_sme', 'dycl', 'pcjeweller', 'prakash', 'ranasug', 'il&fsengg', 'capacite', 'hindnatgls', 'worth', 'hcc', 'nam-india', 'cgpower', 'nifty private bank', 'dynamic_sme', 'ksl', 'kananiind', 'candc'

In [32]:
all_stock_results['jktyre']

[{'Start Date': '2007-11-29',
  'End Date': '2007-12-31',
  'High': 41.0,
  'Low': 25.0,
  'Percentage Move': 64.0,
  'Average Turnover': 1599650.451,
  '20-Day ADR%': 6.799458092518362,
  'ATR': 1.7457412880919954,
  'Distance from 10EMA': 0.8353125481760557,
  'Distance from 20EMA': 0.842901834552908,
  'Distance from 50EMA': 0.511768568121294},
 {'Start Date': '2008-01-01',
  'End Date': '2008-01-29',
  'High': 39.6,
  'Low': 22.05,
  'Percentage Move': 79.59183673469387,
  'Average Turnover': 3778693.292,
  '20-Day ADR%': 8.192275845284659,
  'ATR': 2.6411912333836916,
  'Distance from 10EMA': 5.4191104553764795,
  'Distance from 20EMA': 7.27901774520646,
  'Distance from 50EMA': 9.505256201817613},
 {'Start Date': '2008-02-21',
  'End Date': '2008-03-25',
  'High': 29.6,
  'Low': 17.3,
  'Percentage Move': 71.09826589595376,
  'Average Turnover': 1904331.1979999999,
  '20-Day ADR%': 13.684734622038587,
  'ATR': 3.3738212433842087,
  'Distance from 10EMA': 0.08287848381703711,
  'D

In [33]:
# Initialize an empty list to store each row as a dictionary
flattened_data = []

# Loop through all_stock_results to flatten it
for stock_name, moves in all_stock_results.items():
    for move in moves:
        # Add the stock_name to each move dictionary
        move_with_stock = move.copy()  # Create a copy to avoid modifying the original dictionary
        move_with_stock['Stock Name'] = stock_name
        flattened_data.append(move_with_stock)

# Create a DataFrame from the flattened data
all_stock_results_df = pd.DataFrame(flattened_data)

In [35]:
# Identify numerical columns based on their data types
numerical_columns = all_stock_results_df.select_dtypes(include=['number']).columns

# Round all numerical columns to 2 decimal places
all_stock_results_df[numerical_columns] = all_stock_results_df[numerical_columns].round(2)

In [36]:
all_stock_results_df

,Start Date,End Date,High,Low,Percentage Move,Average Turnover,20-Day ADR%,ATR,Distance from 10EMA,Distance from 20EMA,Distance from 50EMA,Stock Name,Distance from 200EMA
0,2021-12-14,2022-01-11,102.45,66.65,53.71,8.161018e+06,5.10,3.69,6.85,7.73,7.35,intlconv,NaN
1,2020-02-12,2020-03-13,789.00,500.05,57.78,6.126058e+06,2.28,22.54,-28.77,-42.78,-64.75,diamondyd,-101.61
2,2020-05-05,2020-06-03,703.70,435.10,61.73,1.548968e+07,6.69,34.17,-12.04,-28.91,-108.16,diamondyd,-289.07
3,2008-09-24,2008-10-24,825.00,505.00,63.37,2.927733e+09,4.20,36.54,0.43,-4.73,-8.17,bhartiartl,-27.26
4,2008-10-27,2008-11-26,748.00,483.00,54.87,2.370784e+09,10.06,66.54,-130.51,-169.49,-213.85,bhartiartl,-264.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53864,2021-11-25,2022-01-12,653.85,410.00,59.48,NaN,2.49,15.67,43.14,67.35,NaN,bewltd_sme,NaN
53865,2022-01-13,2022-02-11,1087.65,608.00,78.89,1.108930e+07,0.93,25.41,83.81,128.02,238.81,bewltd_sme,NaN
53866,2022-02-14,2022-03-16,1187.20,695.00,70.82,1.339441e+07,3.27,42.03,162.92,259.59,438.65,bewltd_sme,NaN
53867,2022-03-28,2022-04-29,951.00,612.00,55.39,5.835270e+06,4.80,48.53,-31.11,-68.65,-33.74,bewltd_sme,NaN


In [43]:
def add_trading_days_between_high_low(stock_data: pd.DataFrame, results_df: pd.DataFrame) -> None:
    """
    Adds a 'Trading Days' column to results_df, representing the number of trading days between the
    low and high days within each window.
    
    Parameters:
    - stock_data (pd.DataFrame): The stock data DataFrame.
    - results_df (pd.DataFrame): The DataFrame containing moves for a specific stock.
    """
    for idx, row in results_df.iterrows():
        start_date = row['Start Date']
        end_date = row['End Date']
        
        # Filter stock_data to only include the window between start_date and end_date
        window_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] <= end_date)]
        
        # Find the indices of the low and high days within the window
        low_day_index = window_data['Low'].idxmin()
        high_day_index = window_data['High'].idxmax()
        
        # Calculate the number of trading days and update the DataFrame
        trading_days = abs(high_day_index - low_day_index) + 1  # +1 to include both low and high days
        results_df.loc[idx, 'Trading Days'] = trading_days

In [44]:

# Initialize the 'Trading Days' column to 0
all_stock_results_df['Trading Days'] = 0

# Loop through each unique stock name in all_stock_results_df
for stock_name in all_stock_results_df['Stock Name'].unique():
    # Filter the DataFrame for this stock's moves
    stock_moves_df = all_stock_results_df[all_stock_results_df['Stock Name'] == stock_name].copy()
    
    # Load the stock data
    stock_data = pd.read_csv(f"{dir_path}/{stock_name}.csv")
    
    # Update the 'Trading Days' for this stock's moves
    add_trading_days_between_high_low(stock_data, stock_moves_df)
    
    # Update the original all_stock_results_df DataFrame with the new 'Trading Days' data
    all_stock_results_df.update(stock_moves_df)


In [48]:
i = 0
all_stock_results_df.to_csv(f'all_stock_results_df_{i}.csv')

In [46]:
# Convert 'Start Date' to datetime format if it's not already
all_stock_results_df['Start Date'] = pd.to_datetime(all_stock_results_df['Start Date'])

# Extract month and year and create new columns
all_stock_results_df['Start Month'] = all_stock_results_df['Start Date'].dt.month
all_stock_results_df['Start Year'] = all_stock_results_df['Start Date'].dt.year
